# Caso Práctico: Predicciones de Series Temporales empleando Deep Learning.


En este caso práctico veremos cómo entrenar diferentes redes neuronales para resolver la tarea de pronosticar valores futuros de una serie temporal.

El conjunto de datos que emplearemos se conoce como _PJM Hourly Energy Consumption Data_. PJM Interconnection LLC (PJM) es una organización de transmisión regional ubicada en EEUU. Forma parte de la red de Interconexión del Este que opera un sistema de transmisión eléctrica sirve en los estados de Delaware, Illinois, Indiana, Kentucky, Maryland, Michigan, New Jersey, North Carolina, Ohio, Pennsylvania, Tennessee, Virginia, West Virginia, y el Distrito de Columbia.

Estos datos se han recopilado de la web de PJM. El consumo energético corresponde corresponde con el consumo comprendido entre 2001 y 20128. Además,el consumo está en Megavatios.

Para implementar las redes neuronales emplearemos el framework Pytorch. Pytorch es uno de los principales frameworks de código abierto para desarrollar y entrenar modelos de Deep Learning. Ofrece flexibilidad y una interfaz intuitiva que facilita la implementación de diferentes arquitecturas de redes neuronales.

Los modelos que vamos a implementar son los siguientes:

- RNN
- LSTM
- GRU

Para ello, seguiremos los pasos típicos durante el entrenamiento de modelos de ML:

1. Carga de datos: En esta fase cargaremos el conjunto de datos.
1. Visualización de los datos: Visualizaremos los datos para hacernos una idea del aspecto que tienen.
1. Preparación de datos: Realizaremos la preparación necesaria de los datos para alimentar los modelos.
1. Implementación de modelos: Implementaremos cada una de las arquitecturas de redes neuronales mencionadas.
1. Entrenamiento de modelos: Entrenaremos los modelos utilizando los datos de entrenamiento.
1. Evaluación de modelos: Evaluaremos el rendimiento de los modelos utilizando datos de validación y prueba.




# Importación de librerías

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
import holidays
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.offline as pyo
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 1. Carga de datos

El conjunto de datos está almacenado con formato CSV. Empleando la librería de pandas permite cargar los datos de forma sencilla y permite observar las columnas que tenemos en los datos.

In [80]:
df = pd.read_csv('PJME_hourly.csv')
df.head()

,Datetime,PJME_MW
0,2002-12-31 01:00:00,26498.0
1,2002-12-31 02:00:00,25147.0
2,2002-12-31 03:00:00,24574.0
3,2002-12-31 04:00:00,24393.0
4,2002-12-31 05:00:00,24860.0


# 2. Visualización de los datos

Vamos a represetnar los datos para poder hacernos una idea de qué pintan tienen los datos. No vamos a realizar un análisis exhautivo de los datos. Únicamente representaremos las observaciones a lo largo del tiempo para poder observar las peculiariades que presenta a alto nivel.

In [81]:
def plot_dataset(df, title):
    data = []

    value = go.Scatter(
        x=df.index,
        y=df.value,
        mode="lines",
        name="values",
        marker=dict(),
        text=df.index,
        line=dict(color="rgba(0,0,0, 0.3)"),
    )
    data.append(value)

    layout = dict(
        title=title,
        xaxis=dict(title="Date", ticklen=5, zeroline=False),
        yaxis=dict(title="Value", ticklen=5, zeroline=False),
    )

    fig = dict(data=data, layout=layout)
    iplot(fig)

Podemos observar 2 estacionalidaes muy marcadas. Una en los periodos estivales y otra en los invernales. Por tanto, tenemos 2 estacionalidades anuales. También observamos que tenemos máximos en el consumo en el mes de agosto y en el mes de Febrero, que suelen ser los meses más caluroso y más frío, respectivamente.

In [82]:
df = df.set_index(['Datetime'])
df = df.rename(columns={'PJME_MW': 'value'})

df.index = pd.to_datetime(df.index)
if not df.index.is_monotonic_increasing:
    df = df.sort_index()

plot_dataset(df, title='PJME: Consumo de energía estimado en Megawatts (MW)')

Output hidden; open in https://colab.research.google.com to view.

# 3. Preparación de los datos

En esta sección realizaremos la preparación de los datos.

Lo primero que haremos es generar el vector de time-delay que será el vector de entrada que procesaran nuestros modelos. Recordemos que este vector nos permite abordar el problema de pronosticar valores futuros de consumo como si fuera un problema típico de regresión.

Es importante escoger adecuadamente las observaciones pasadas que empleamos para realizar la predicción. Otro factor clave es la selección del horizonte de pronóstico.

In [83]:
def create_time_delay_embedding(data, horizon=6, final_lag=24, column="y", column_index=None):
    df = pd.DataFrame()
    df["y"] = data[column].copy()
    for i in range(horizon, final_lag+1):
        df["lag_{}".format(i)] = data[column].shift(i)
    if column_index is None:
        df.index = data.index
    else:
        df.index = data[column_index]
    return df

In [84]:
final_lag = 100
horizon = 1

df_time_delay = create_time_delay_embedding(df, horizon=horizon, final_lag=final_lag, column="value")
df_time_delay = df_time_delay.dropna()

<ipython-input-83-58f1fd0af011>:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



Ahora podemos crear predictores que ayuden a nuestro modelo a realizar las predicciones. En este caso podemos extraer la siguiente información para cada file de nueestro conjunto de datos:

- Hora
- Día
- Mes
- Día de la semana

In [85]:
df_features = (
                df_time_delay
                .assign(hour = df_time_delay.index.hour)
                .assign(day = df_time_delay.index.day)
                .assign(month = df_time_delay.index.month)
                .assign(day_of_week = df_time_delay.index.dayofweek)
                # .assign(week_of_year = df.index.week)
              )

In [86]:
df_features.head()

,y,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_95,lag_96,lag_97,lag_98,lag_99,lag_100,hour,day,month,day_of_week
Datetime,,,,,,,,,,,,,,,,,,,,,
2002-01-05 05:00:00,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,33202.0,35368.0,36762.0,37539.0,...,28654.0,28057.0,27899.0,28357.0,29265.0,30393.0,5,5,1,5
2002-01-05 06:00:00,27399.0,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,33202.0,35368.0,36762.0,...,29308.0,28654.0,28057.0,27899.0,28357.0,29265.0,6,5,1,5
2002-01-05 07:00:00,28557.0,27399.0,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,33202.0,35368.0,...,29595.0,29308.0,28654.0,28057.0,27899.0,28357.0,7,5,1,5
2002-01-05 08:00:00,29709.0,28557.0,27399.0,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,33202.0,...,29943.0,29595.0,29308.0,28654.0,28057.0,27899.0,8,5,1,5
2002-01-05 09:00:00,31241.0,29709.0,28557.0,27399.0,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,...,30692.0,29943.0,29595.0,29308.0,28654.0,28057.0,9,5,1,5


Una forma de ayudar a nuestro modelo a lidiar con factores exógenos que influyen en el valor de nuestra serie temporal es indicar si la observación a predecir corresponde con un día festivo.

Para ello, podemos emplear el calendario vacacional de EEUU e incluirlo como predictor en nuestra serie.

Emplearemos una variable binaria (puede tomar 0 o 1 como valor) para indicar si la observación corresponde con un día festivo (_is_holiday = 1_) o por el contrario no lo es (_is_holiday = 1_).

In [87]:
us_holidays = holidays.US()

def is_holiday(date):
    date = date.replace(hour = 0)
    return 1 if (date in us_holidays) else 0

def add_holiday_col(df, holidays):
    return df.assign(is_holiday = df.index.to_series().apply(is_holiday))


df_features = add_holiday_col(df_features, us_holidays)
df_features.head(3)

,y,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_96,lag_97,lag_98,lag_99,lag_100,hour,day,month,day_of_week,is_holiday
Datetime,,,,,,,,,,,,,,,,,,,,,
2002-01-05 05:00:00,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,33202.0,35368.0,36762.0,37539.0,...,28057.0,27899.0,28357.0,29265.0,30393.0,5,5,1,5,0
2002-01-05 06:00:00,27399.0,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,33202.0,35368.0,36762.0,...,28654.0,28057.0,27899.0,28357.0,29265.0,6,5,1,5,0
2002-01-05 07:00:00,28557.0,27399.0,26822.0,26669.0,27034.0,27501.0,28635.0,30924.0,33202.0,35368.0,...,29308.0,28654.0,28057.0,27899.0,28357.0,7,5,1,5,0


Ahora es el momento de realizar la partición de nuestros datos indicando cuál es nuestra variable objetivo y nuestras variables de entrada.

Además, partimos nuestro dataset en 2 subcojnjuntos, uno para realizar el entrenamiento y otro para realizar la validación del modelo entrenado.

In [88]:
def feature_label_split(df, target_col):
    y = df[[target_col]]
    X = df.drop(columns=[target_col])
    return X, y

def train_val_test_split(df, target_col, test_ratio):
    val_ratio = test_ratio / (1 - test_ratio)
    X, y = feature_label_split(df, target_col)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle=False)
    return X_train, X_val, X_test, y_train, y_val, y_test



def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df_features, 'y', 0.2)

Después realizamos el escalado de los datos para ayudar a nuestros modelos durante la fase de entrenamiento ya que estos mejoran su rendimiento cuando procesan vectores con números de pequeña magnitud.

In [89]:
scaler = get_scaler('minmax')
X_train_arr = scaler.fit_transform(X_train)
X_val_arr = scaler.transform(X_val)
X_test_arr = scaler.transform(X_test)

y_train_arr = scaler.fit_transform(y_train)
y_val_arr = scaler.transform(y_val)
y_test_arr = scaler.transform(y_test)

Ya casi hemos finalizado la etapa de preparación de datos. El último paso es crear un objeto _Dataset_ de Pytorch para poder alimentar los modelos.

El objeto _Dataset_ en Pytorch es una clase que proporciona una interfaz uniforme para acceder a los datos y sus etiquetas. Para nuestro caso, crearemos un objeto Dataset que contenga las secuencias de datos de entrada y sus correspondientes etiquetas de salida.

Pytorch ofrece la posibilidad de emplear funciones por defecto que crean este objeto a partir de los datos que tenemos. En muchas ocasiones estas funciones por defecto se quedan cortas y es necesario implementar una clase personalizada.

Para hacer esto, primero necesitamos definir una clase personalizada que herede de la clase base Dataset de Pytorch. En esta clase personalizada, implementaremos dos métodos principales:

__init__: En este método, inicializaremos los datos de entrada y salida que queremos alimentar a nuestros modelos. Esto puede implicar dividir la serie temporal en secuencias de entrada y etiquetas de salida, teniendo en cuenta el tamaño de la ventana temporal y cualquier otra consideración relevante para la arquitectura del modelo.

__getitem__: Este método nos permitirá acceder a un elemento específico del conjunto de datos. Aquí, devolveremos una secuencia de entrada junto con su correspondiente etiqueta de salida.

Además de estos dos métodos, también podemos implementar el método len para obtener la longitud del conjunto de datos.


Sin embargo, en nuestro caso nos es suficiente con emplear la clase _TensorDataset_ y posteriormente crear el _DataLoader_.

In [90]:
batch_size = 64

train_features = torch.Tensor(X_train_arr)
train_targets = torch.Tensor(y_train_arr)
val_features = torch.Tensor(X_val_arr)
val_targets = torch.Tensor(y_val_arr)
test_features = torch.Tensor(X_test_arr)
test_targets = torch.Tensor(y_test_arr)

train = TensorDataset(train_features, train_targets)
val = TensorDataset(val_features, val_targets)
test = TensorDataset(test_features, test_targets)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)

# 4. Entrenamiento y validación de modelos

En esta fase realizaremos el entrenamiento de los modelos.

El primer paso es definir la arquitectura de cada uno de los modelos. Para implementar los modelos mediante Pytorch, necesitamos definir las arquitecturas de la RNN, LSTM y GRU. Para cada modelo, definiremos una clase personalizada que herede de la clase base de PyTorch para modelos, generalmente llamada __nn.Module__.

En cada una de estas clases personalizadas, definiremos la estructura de la red neuronal, especificando las capas y cualquier operación necesaria. Esto incluirá la capa de entrada, una o varias capas recurrentes (dependiendo del tipo de modelo), y posiblemente capas de salida adicionales, como capas completamente conectadas.

Luego, implementaremos el método forward, que especifica cómo se propagan hacia adelante los datos a través de la red neuronal. En este método, pasaremos los datos de entrada a través de las capas definidas y devolveremos la salida del modelo.

## RNN
La clase RNNModel define una arquitectura de red neuronal recurrente (RNN) para la predicción de observaciones futuraas. A continuación, se explican los componentes principales de esta arquitectura:

Inicialización: En el método __init__, se definen los parámetros principales de la red, como el número de unidades en la capa de entrada (input_dim), el número de unidades en cada capa oculta (hidden_dim), el número de capas en la red (layer_dim), el número de unidades en la capa de salida (output_dim), y la probabilidad de dropout para las capas (dropout_prob).

Capas RNN: Se define la capa RNN utilizando nn.RNN. Esta capa tiene como entrada el tamaño de la entrada (input_dim), el tamaño de la capa oculta (hidden_dim), el número de capas (layer_dim), y la probabilidad de dropout (dropout_prob). Se establece batch_first=True para que la entrada tenga la forma (tamaño del batch, longitud de secuencia, input_dim).

Capas Fully-Connected (Completamente Conectadas): Se define una capa completamente conectada (nn.Linear) que toma la salida de la capa RNN y la transforma en la forma deseada de salida (output_dim).

Propagación hacia adelante: En el método forward, se realiza la propagación hacia adelante de la red. Se inicializa el estado oculto h0 con ceros y se pasa como entrada a la capa RNN junto con la entrada x. La salida de la capa RNN se reorganiza para que tenga la forma adecuada (tamaño del batch, longitud de secuencia, tamaño oculto) y luego se pasa a través de la capa completamente conectada para obtener la salida final.

En resumen, esta arquitectura de red RNN tiene una estructura básica con una capa RNN seguida de una capa completamente conectada para la predicción de secuencias.

In [91]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        """ Inicializa una instancia de RNN.

        Args:
            input_dim (int): El número de nodos en la capa de entrada
            hidden_dim (int): El número de nodos en cada capa oculta
            layer_dim (int): El número de capas en la red
            output_dim (int): El número de nodos en la capa de salida
            dropout_prob (float): La probabilidad de dropout para las capas

        """
        super(RNNModel, self).__init__()

        # Definición del número de capas y los nodos en cada capa
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # Capas RNN
        self.rnn = nn.RNN(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )
        # Capas Fully-Connected
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """Realiza la propagación hacia adelante.

        Args:
            x (torch.Tensor): Tensor de entrada de forma (tamaño del batch, longitud de secuencia, input_dim)

        Returns:
            torch.Tensor: Tensor de salida de forma (tamaño del bacth, output_dim)

        """
        # Inicializando el estado oculto para la primera entrada con ceros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim,device=x.device).requires_grad_()

        # Propagación hacia adelante pasando la entrada y el estado oculto al modelo
        out, h0 = self.rnn(x, h0.detach())

        # Reorganizando las salidas en la forma (tamaño del batch, longitud de secuencia, tamaño oculto)
        # para que pueda encajar en la capa completamente conectada
        out = out[:, -1, :]

        # Convertir el estado final a nuestra forma de salida deseada (tamaño del batch, output_dim)
        out = self.fc(out)
        return out

## LSTM

La clase LSTMModel define una arquitectura de red neuronal de tipo Long Short-Term Memory (LSTM) para la predicción de secuencias.

Inicialización: En el método __init__, se definen los parámetros principales de la red, como el número de nodos en la capa de entrada (input_dim), el número de nodos en cada capa oculta (hidden_dim), el número de capas en la red (layer_dim), el número de nodos en la capa de salida (output_dim), y la probabilidad de dropout para las capas (dropout_prob).

Capas LSTM: Se define la capa LSTM utilizando nn.LSTM. Esta capa tiene como entrada el tamaño de la entrada (input_dim), el tamaño de la capa oculta (hidden_dim), el número de capas (layer_dim), y la probabilidad de dropout (dropout_prob). Se establece batch_first=True para que la entrada tenga la forma (tamaño del batch, longitud de secuencia, input_dim).

Capa Fully-Connected (Completamente Conectada): Se define una capa completamente conectada (nn.Linear) que toma la salida de la capa LSTM y la transforma en la forma deseada de salida (output_dim).

Propagación hacia adelante: En el método forward, se realiza la propagación hacia adelante de la red. Se inicializan los estados ocultos h0 y de celda c0 con ceros y se pasan como entrada a la capa LSTM junto con la entrada x. La salida de la capa LSTM se reorganiza para que tenga la forma adecuada (tamaño del batch, longitud de secuencia, tamaño oculto) y luego se pasa a través de la capa completamente conectada para obtener la salida final.

In [92]:
class LSTMModel(nn.Module):
    """Clase LSTMModel que extiende la clase nn.Module y funciona como constructor de LSTMs.

       La clase LSTMModel inicia un módulo LSTM basado en la clase nn.Module de PyTorch.
       Tiene solo dos métodos, a saber, init() y forward(). Mientras que el método init()
       inicia el modelo con los parámetros de entrada dados, el método forward()
       define cómo se debe calcular la propagación hacia adelante.
       Dado que PyTorch define automáticamente la retropropagación, no es necesario
       definir el método de retropropagación.

       Atributos:
           hidden_dim (int): El número de nodos en cada capa oculta
           layer_dim (int): El número de capas en la red
           lstm (nn.LSTM): El modelo LSTM construido con los parámetros de entrada.
           fc (nn.Linear): La capa completamente conectada para convertir el estado final
                           de las LSTMs a nuestra forma de salida deseada.

    """
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        """El método __init__ que inicia una instancia de LSTM.

        Args:
            input_dim (int): El número de nodos en la capa de entrada
            hidden_dim (int): El número de nodos en cada capa oculta
            layer_dim (int): El número de capas en la red
            output_dim (int): El número de nodos en la capa de salida
            dropout_prob (float): La probabilidad de dropout para las capas

        """
        super(LSTMModel, self).__init__()

        # Definiendo el número de capas y los nodos en cada capa
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim

        # Capas LSTM
        self.lstm = nn.LSTM(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Capa completamente conectada
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """El método forward toma el tensor de entrada x y realiza la propagación hacia adelante

        Args:
            x (torch.Tensor): El tensor de entrada de forma (tamaño del batch, longitud de secuencia, input_dim)

        Returns:
            torch.Tensor: El tensor de salida de forma (tamaño del batch, output_dim)

        """
        # Inicializando el estado oculto para la primera entrada con ceros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device).requires_grad_()
        # Inicializando el estado de celda para la primera entrada con ceros
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device).requires_grad_()

        # Necesitamos desvincular porque estamos haciendo retropropagación truncada a través del tiempo (BPTT)
        # Si no lo hacemos, retrocederemos hasta el principio incluso después de pasar por otro batch
        # Propagación hacia adelante pasando la entrada, el estado oculto y el estado de celda al modelo
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Reorganizando las salidas en la forma de (tamaño del batch, longitud de secuencia, tamaño oculto)
        # para que pueda encajar en la capa completamente conectada
        out = out[:, -1, :]

        # Convertir el estado final a nuestra forma de salida deseada (tamaño del batch, output_dim)
        out = self.fc(out)

        return out


## GRU
La clase GRUModel define una arquitectura de red neuronal de tipo Gated Recurrent Unit (GRU) para la predicción de secuencias.

Inicialización: En el método __init__, se definen los parámetros principales de la red, como el número de nodos en la capa de entrada (input_dim), el número de nodos en cada capa oculta (hidden_dim), el número de capas en la red (layer_dim), el número de nodos en la capa de salida (output_dim), y la probabilidad de dropout para las capas (dropout_prob).

Capas GRU: Se define la capa GRU utilizando nn.GRU. Esta capa tiene como entrada el tamaño de la entrada (input_dim), el tamaño de la capa oculta (hidden_dim), el número de capas (layer_dim), y la probabilidad de dropout (dropout_prob). Se establece batch_first=True para que la entrada tenga la forma (tamaño del batch, longitud de secuencia, input_dim).

Capa Fully-Connected (Completamente Conectada): Se define una capa completamente conectada (nn.Linear) que toma la salida de la capa GRU y la transforma en la forma deseada de salida (output_dim).

Propagación hacia adelante: En el método forward, se realiza la propagación hacia adelante de la red. Se inicializa el estado oculto h0 con ceros y se pasa como entrada a la capa GRU junto con la entrada x. La salida de la capa GRU se reorganiza para que tenga la forma adecuada (tamaño del batch, longitud de secuencia, tamaño oculto) y luego se pasa a través de la capa completamente conectada para obtener la salida final.

En resumen, esta arquitectura de red GRU tiene una estructura básica con una capa GRU seguida de una capa completamente conectada para la predicción de secuencias.

In [102]:
class GRUModel(nn.Module):
    """La clase GRUModel extiende la clase nn.Module y funciona como un constructor para GRUs.

       La clase GRUModel inicia un módulo GRU basado en la clase nn.Module de PyTorch.
       Tiene solo dos métodos, a saber, init() y forward(). Mientras que el método init()
       inicia el modelo con los parámetros de entrada dados, el método forward()
       define cómo se debe calcular la propagación hacia adelante.
       Dado que PyTorch define automáticamente la retropropagación, no es necesario
       definir el método de retropropagación.

       Atributos:
           hidden_dim (int): El número de nodos en cada capa oculta
           layer_dim (int): El número de capas en la red
           gru (nn.GRU): El modelo GRU construido con los parámetros de entrada.
           fc (nn.Linear): La capa completamente conectada para convertir el estado final
                           de las GRUs a nuestra forma de salida deseada.

    """
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, dropout_prob):
        """El método __init__ inicia una instancia de GRU.

        Args:
            input_dim (int): El número de nodos en la capa de entrada
            hidden_dim (int): El número de nodos en cada capa oculta
            layer_dim (int): El número de capas en la red
            output_dim (int): El número de nodos en la capa de salida
            dropout_prob (float): La probabilidad de dropout para las capas

        """
        super(GRUModel, self).__init__()

        # Definición del número de capas y los nodos en cada capa
        self.layer_dim = layer_dim
        self.hidden_dim = hidden_dim

        # Capas GRU
        self.gru = nn.GRU(
            input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob
        )

        # Capa completamente conectada
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        """El método forward toma el tensor de entrada x y realiza la propagación hacia adelante

        Args:
            x (torch.Tensor): El tensor de entrada de forma (tamaño del batch, longitud de secuencia, input_dim)

        Returns:
            torch.Tensor: El tensor de salida de forma (tamaño del batch, output_dim)

        """
        # Inicializando el estado oculto para la primera entrada con ceros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=x.device).requires_grad_()

        # Propagación hacia adelante pasando la entrada y el estado oculto al modelo
        out, _ = self.gru(x, h0.detach())

        # Reorganizando las salidas en la forma de (tamaño del batch, longitud de secuencia, tamaño oculto)
        # para que pueda encajar en la capa completamente conectada
        out = out[:, -1, :]

        # Convertir el estado final a nuestra forma de salida deseada (tamaño del batch, output_dim)
        out = self.fc(out)

        return out


## Definición de funciones útiles para el entrenamiento y validación del modelo

En esta sección se definen funciones que nos ayudarán en el entrenamineto de redes neuronales. A continuación se presenta una breve descripción para cada una de estas funciones:

- _get_model_: función que devuelve una instancia de un modelo de red neuronal recurrente específico, como RNNModel, LSTMModel o GRUModel, dependiendo del tipo de modelo especificado y los parámetros del modelo proporcionados como entrada.
- _optimization class_: Optimization es una clase auxiliar que proporciona un marco para entrenar y validar los modelos.
- _General_Setting_: configura y entrena uno de los modelos que ha sido especificado a partir del nombre del modelo como argumento.
- _inverse_transform_: Realiza la transformación inversa del escalado.
- _format_predictions_: se encarga de formatear las predicciones realizadas por el modelo seleccionado y los valores reales correspondientes.
- _calculate_metrics_: Calcula el error de las predicciones generadas por el modelo seleccionado.
- _build_baseline_model_: Crea un modelo sencillo de regresión lineal para comparar las predicciones del modelo seleccionado.
- _plot_predictions_: Representa las predicciones generadas junto con los valores reales


In [93]:
def get_model(model, model_params):
    models = {
        "rnn": RNNModel,
        "lstm": LSTMModel,
        "gru": GRUModel,
    }
    return models.get(model.lower())(**model_params)

In [103]:
class Optimization:
    """La clase Optimization es una clase auxiliar que permite entrenar, validar y predecir.

    Optimization es una clase auxiliar que toma el modelo, la función de pérdida, la función de optimización
    el programador de aprendizaje (opcional), la detención anticipada (opcional) como entradas. A cambio,
    proporciona un marco para entrenar y validar los modelos, y para predecir valores futuros
    basado en los modelos.

    Atributos:
        model (RNNModel, LSTMModel, GRUModel): Clase del modelo creada para el tipo de RNN
        loss_fn (torch.nn.modules.Loss): Función de pérdida para calcular las pérdidas
        optimizer (torch.optim.Optimizer): Función de optimización para optimizar la función de pérdida
        train_losses (list[float]): Los valores de pérdida del entrenamiento
        val_losses (list[float]): Los valores de pérdida de la validación
        last_epoch (int): El número de épocas que el modelo ha sido entrenado
    """
    def __init__(self, model, loss_fn, optimizer):
        """
        Args:
            model (RNNModel, LSTMModel, GRUModel): Clase del modelo creada para el tipo de RNN
            loss_fn (torch.nn.modules.Loss): Función de pérdida para calcular las pérdidas
            optimizer (torch.optim.Optimizer): Función de optimización para optimizar la función de pérdida
        """
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []

    def train_step(self, x, y):
        """El método train_step completa un paso de entrenamiento.

        Dados los tensores de características (x) y los valores objetivo (y), el método completa
        un paso del entrenamiento. Primero, activa el modo de entrenamiento para habilitar la retropropagación.
        Después de generar valores predichos (yhat) haciendo propagación hacia adelante, calcula
        las pérdidas usando la función de pérdida. Luego, calcula los gradientes haciendo
        retropropagación y actualiza los pesos llamando a la función step().

        Args:
            x (torch.Tensor): Tensor de características para entrenar un paso
            y (torch.Tensor): Tensor de valores objetivo para calcular las pérdidas

        """
        # Establece el modelo en modo de entrenamiento
        self.model.train()

        # Realiza predicciones
        yhat = self.model(x)

         # Calcula la pérdida
        loss = self.loss_fn(y, yhat)

        # Calcula los gradientes
        loss.backward()

        # Actualiza los parámetros y pone a cero los gradientes
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Devuelve la pérdida
        return loss.item()

    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):
        """El método train realiza el entrenamiento del modelo

        El método toma los DataLoaders para los conjuntos de datos de entrenamiento y validación, el tamaño de batch para
        el entrenamiento en mini-batches, el número de épocas para entrenar y el número de características como entradas.
        Luego, lleva a cabo el entrenamiento llamando iterativamente al método train_step durante
        n_epochs veces. Si la detención anticipada está habilitada, entonces comprueba la condición de parada
        para decidir si el entrenamiento debe detenerse antes de n_epochs pasos. Finalmente, guarda
        el modelo en una ruta de archivo designada.

        Args:
            train_loader (torch.utils.data.DataLoader): DataLoader que almacena los datos de entrenamiento
            val_loader (torch.utils.data.DataLoader): DataLoader que almacena los datos de validación
            batch_size (int): Tamaño de batch para el entrenamiento en mini-batches
            n_epochs (int): Número de épocas, es decir, pasos de entrenamiento, para entrenar
            n_features (int): Número de columnas de características

        """
        model_path = f'{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'

        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([batch_size, -1, n_features])
                y_batch = y_batch
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features])
                    y_val = y_val
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch % 50 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )

        torch.save(self.model.state_dict(), model_path)

    def evaluate(self, test_loader, batch_size=1, n_features=1):
        """El método evaluate realiza la evaluación del modelo

        El método toma DataLoaders para el conjunto de datos de prueba, el tamaño de batch para la prueba en mini-batches,
        y el número de características como entradas. Similar a la validación del modelo, itera
        predice los valores objetivo y calcula las pérdidas. Luego, devuelve dos listas que
        contienen las predicciones y los valores reales.

        Nota:
            Este método asume que la predicción del paso anterior está disponible en
            el momento de la predicción, y solo hace una predicción de un paso hacia el futuro.

        Args:
            test_loader (torch.utils.data.DataLoader): DataLoader que almacena los datos de prueba
            batch_size (int): Tamaño de batch para la prueba en mini-batches
            n_features (int): Número de columnas de características

        Returns:
            list[float]: Los valores predichos por el modelo
            list[float]: Los valores reales en el conjunto de prueba.

        """
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features])
                y_test = y_test
                self.model.eval()
                yhat = self.model(x_test)
                yhat=yhat.cpu().data.numpy()
                predictions.append(yhat)
                y_test=y_test.cpu().data.numpy()
                values.append(y_test)

        return predictions, values

    def plot_losses(self):
        """El método plot_losses traza los valores de pérdida calculados
        para entrenamiento y validación
        """
        plt.style.use('ggplot')
        plt.figure(figsize=(10,5))
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [104]:
def General_Settings(model_name):

        input_dim = len(X_train.columns)
        output_dim = 1
        hidden_dim = 64
        layer_dim = 3
        batch_size = 64
        dropout = 0.2
        n_epochs = 20
        learning_rate = 1e-3
        weight_decay = 1e-6

        model_params = {'input_dim': input_dim,
                        'hidden_dim' : hidden_dim,
                        'layer_dim' : layer_dim,
                        'output_dim' : output_dim,
                        'dropout_prob' : dropout}
        model = get_model(model_name, model_params)
        loss_fn = nn.MSELoss(reduction="mean")
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
        opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
        opt.plot_losses()

        predictions, values = opt.evaluate(
            test_loader_one,
            batch_size=1,
            n_features=input_dim
        )
        return predictions,values


In [105]:
def inverse_transform(scaler, df, columns):
    for col in columns:
        df[col] = scaler.inverse_transform(df[col])
    return df


def format_predictions(predictions, values, df_test, scaler, col_name="y"):
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()
    df_result = pd.DataFrame(data={col_name: vals, "prediction": preds}, index=df_test.head(len(vals)).index)
    df_result = df_result.sort_index()
    df_result = inverse_transform(scaler, df_result, [[col_name, "prediction"]])
    return df_result

In [106]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(df, col_name="y"):
    result_metrics = {'mae' : mean_absolute_error(df[col_name], df.prediction),
                      'rmse' : mean_squared_error(df[col_name], df.prediction) ** 0.5,
                      'r2' : r2_score(df[col_name], df.prediction)}

    print("Mean Absolute Error:       ", result_metrics["mae"])
    print("Root Mean Squared Error:   ", result_metrics["rmse"])
    print("R^2 Score:                 ", result_metrics["r2"])
    return result_metrics

In [107]:
def build_baseline_model(df, test_ratio, target_col):
    X, y = feature_label_split(df, target_col)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_ratio, shuffle=False
    )
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)

    result = pd.DataFrame(y_test)
    result["prediction"] = prediction
    result = result.sort_index()

    return result

In [108]:
plt.style.use('ggplot')

def plot_predictions(df_result, df_baseline):
    data = []

    value = go.Scatter(
        x=df_result.index,
        y=df_result.y,
        mode="lines",
        name="values",
        marker=dict(),
        text=df_result.index,
        line=dict(color="rgba(0,0,0, 0.3)"),
    )
    data.append(value)

    baseline = go.Scatter(
        x=df_baseline.index,
        y=df_baseline.prediction,
        mode="lines",
        line={"dash": "dot"},
        name='linear regression',
        marker=dict(),
        text=df_baseline.index,
        opacity=0.8,
    )
    data.append(baseline)

    prediction = go.Scatter(
        x=df_result.index,
        y=df_result.prediction,
        mode="lines",
        line={"dash": "dot"},
        name='predictions',
        marker=dict(),
        text=df_result.index,
        opacity=0.8,
    )
    data.append(prediction)

    layout = dict(
        title="Predictions vs Actual Values for the dataset",
        xaxis=dict(title="Time", ticklen=5, zeroline=False),
        yaxis=dict(title="Value", ticklen=5, zeroline=False),
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

## Entrenamiento y validación de la RNN

In [109]:
# model_name=Enter model name 'lstm','rnn','gru'
model_name='rnn'
predictions, values=General_Settings(model_name)
df_result=format_predictions(predictions, values, X_test, scaler)
print(calculate_metrics(df_result))
df_baseline = build_baseline_model(df_features, 0.2, 'y')
print(calculate_metrics(df_baseline))
plot_predictions(df_result, df_baseline)

Output hidden; open in https://colab.research.google.com to view.

## Entrenamiento y validación de la LSTM

In [101]:
model_name='lstm'
predictions, values=General_Settings(model_name)
df_result=format_predictions(predictions, values, X_test, scaler)
print(calculate_metrics(df_result))
df_baseline = build_baseline_model(df_features, 0.2, 'y')
print(calculate_metrics(df_baseline))
plot_predictions(df_result, df_baseline)

Output hidden; open in https://colab.research.google.com to view.

## Entrenamiento y validación de la GRU

In [74]:
# model_name=Enter model name 'lstm','rnn','gru'
model_name='gru'
predictions, values=General_Settings(model_name)
df_result=format_predictions(predictions, values, X_test, scaler)
print(calculate_metrics(df_result))
df_baseline = build_baseline_model(df_features, 0.2, 'y')
print(calculate_metrics(df_baseline))
plot_predictions(df_result, df_baseline)

Output hidden; open in https://colab.research.google.com to view.